## <center>Calculations for simple 2-state analysis with RMSD-based transition interfaces</center>

##### MUST SHARE KERNEL WITH NOTEBOOK `00-load-data.ipynb`

#### 1. Setup for a small parameter sweep to find good interface placement

------------------------

In [16]:
# DEFINE data organization for this analysis
datasets = {
    # NO UNDERSCORES in keys here
    #  - messes up processing in calculate_observed_rates
    label_longtraj: traj_rmsd_df[label_heavies],
    # more : others)
}

# USER-DEFINED RMSD interfaces
#  - look at notebook "10-rmsd" and pick smart numbers...
#  - choose most-distinguishing RMSD feature
#  - the heavy-atom RMSD has more clearly separated values than Calpha
ifcs_2state_1 = [3.9, 4.4]
ifcs_2state_2 = [4.1, 4.4]
ifcs_3state = [4.1, 4.4, 5.0, 5.5]

pars_2state = {
    "min_residence": [1, 3, 10, 25, 50, 100, 250, 500, 1000],
    "interfaces": [ifcs_2state_1, ifcs_2state_2],
}

# generate a useful identifier for each test parameter set
parset_key = lambda parset: "_".join(
    ["{0}-{1}".format(p, v) for p, v in parset.items()]
)

splitsteps = lambda k: k.split("-")[-2].split("_")[0]
parsets = aswa_tools.mix_parameters(pars_2state)

print(
    colorama.Back.CYAN
    + "Using RMSD data for transition calculations for interface {}".format(
        ifcs_2state_1
    )
)

Using RMSD data for transition calculations for interface [3.9, 4.4]


#### 2. Helpers for saving rate along incremented trajectory, transitions in certain increment of trajectory
----------------------------

In [17]:
incr_length = 200
print(
    colorama.Back.CYAN
    + "Data will be collected in %d-step increments on the trajectory data"
    % incr_length
)

_n_incr = lambda data, incr_length: int(
    (len(data) / incr_length) + int(bool(len(data) % incr_length))
)

n_incr = {dk: _n_incr(datasets[dk], incr_length) for dk in datasets}

# FOR FINDING transitions that occurred between some step numbers i1 and i2
_is_between = lambda v, i1, i2: v > i1 and v < i2
_max_under = lambda td, i1, i2: max(
    [i + 1 if _is_between(fn, i1, i2) else -1 for i, fn in enumerate(td)]
)
_min_over = lambda td, i1, i2: min(
    [i if _is_between(fn, i1, i2) else len(td) for i, fn in enumerate(td)]
)
_relevant_transitions = lambda td, i1, i2: td[
    _min_over(td, i1, i2) : _max_under(td, i1, i2)
]

Data will be collected in 200-step increments on the trajectory data


#### 3. Helpers for later when plotting rate plots
------------

In [18]:
# This stuff makes complicated plot...
# Values to help with extra plot objects
i_frame = 6000
n_frames = 9000
interfacecolor = "ghostwhite"
shading_min = 0.2
shading_max = 8
arrow_linewidth = 6
arrow_length = 0.4

timeseriescolor = [
    r + s for r, s in zip(matplotlib.colors.to_rgba("darkcyan"), (0.25, 0.25, 0.25, 0))
]

-------------
-------------

#### 4. CALCULATE Timepoints with State Transitions AND the Discrete (state) Trajectories
--------------

In [19]:
def print_colorama(message, cformat):
    if len(cformat < 1):
        print("Why not just print plain")
    if len(cformat) == 1:
        target = []
        # TODO FIXME USEME and bring to front!!


print(colorama.Back.CYAN + "Calculating transition locations")
# DATA WILL BE STORED HERE
# THIS ONE TRACKS THE BEST TRANSITION POINTS FOR LATER CALCULATIONS
transition_data = dict()  # pandas.DataFrame()

for dataname, dataset in datasets.items():

    # display("On to dataset: %s"%dataname)

    parnames, parset = list(zip(*parsets.items()))

    for pars in zip(*parset):

        # display("Using parameters: {}".format(pars))

        _pars = {k: v for k, v in list(zip(parnames, pars))}

        transition_data[dataname + "_" + parset_key(_pars)] = [
            aswa_tools.count_changes(dataset, **_pars)
        ]

    # clear_output(wait=True)

# THIS ONE TRACKS THE STATE WHERE IT WAS DETERMINED A TRANSITION HAPPENED
# which isn't useful in practice...
_transition_data = dict()  # pandas.DataFrame()

for dataname, dataset in datasets.items():
    parnames, parset = list(zip(*parsets.items()))
    for pars in zip(*parset):
        _pars = {k: v for k, v in list(zip(parnames, pars))}
        parkey = dataname + "_" + parset_key(_pars)
        _pars.update(dict(use_initial_entry=False))
        _transition_data[parkey] = [aswa_tools.count_changes(dataset, **_pars)]

lookatme = "Single Trajectory_min_residence-50_interfaces-[4.0, 4.4]"

print("Creating the discrete-state trajectories using transition locations")
# DTRAJS
the_dtrajs = {
    dataname: aswa_tools.dtraj_from_changepoints(
        transition_data[dataname], traj_rmsd_df[label_timesteps].shape[0]
    )
    for dataname in transition_data
}

# THIS WAS FOUND OUT BY INSPECTION AND ADDED POST-FACTO
bestone = match_keys(match_keys(list(the_dtrajs), "100_"), "3.9")[0]
print("The 'best' data for analysis is currently set")
print("to the transitions ID'd using these parameters:")
print(colorama.Back.RESET + colorama.Fore.LIGHTBLUE_EX + "%s" % bestone)

Calculating transition locations
Creating the discrete-state trajectories using transition locations
The 'best' data for analysis is currently set
to the transitions ID'd using these parameters:
Single Trajectory_min_residence-100_interfaces-[3.9, 4.4]


#### 5. CALCULATE Observed Event Frequencies
----------------

In [20]:
print(
    colorama.Back.CYAN
    + "calculating observed rate of events from each set of transitions"
)
# Simple: Total observations / Total MD time
observed_rates = aswa_tools.calculate_observed_rates(
    transition_data, incr_length, n_incr, step_per_ns
)

print(
    "aligning timestamps to get a nice axis with MD time vs. rate w/ units: %s"
    % label_timesteps
)
observed_rates.update(
    {
        label_timesteps: {
            dk: pandas.Series(
                [ni * incr_length / step_per_ns[dk] / 1000 for ni in range(n_incr[dk])]
            )
            for dk in datasets
        }
    }
)

longtraj_2states = match_keys(match_keys(observed_rates, label_longtraj), "3.9, 4.4")

calculating observed rate of events from each set of transitions
aligning timestamps to get a nice axis with MD time vs. rate w/ units: Time [μs]


#### $\infty$ DONE
-------

In [21]:
print(f"{dn1}  DONE LOADING DATA  ")

  DONE LOADING DATA  


--------
--------